# RL Trading Model Training & Evaluation

This notebook demonstrates how to train a reinforcement learning (RL) trading model using your full feature pipeline and visualize the results. It loads historical price data, extracts features, trains a PPO agent, and evaluates performance.

In [1]:
# ================================================
# 🔧 SETUP - Add src to Python Path
# ================================================

import sys
import os

# Add src directory to Python path so 'core' module can be found
project_root = os.getcwd()
src_path = os.path.join(project_root, 'src')

if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"✅ Added to Python path: {src_path}")
else:
    print(f"✅ Already in path: {src_path}")

# Verify
print(f"📂 Working directory: {project_root}")
print(f"🔍 Python will search for modules in: {src_path}")
print("=" * 50)

✅ Added to Python path: d:\Dev\trading-bot\src
📂 Working directory: d:\Dev\trading-bot
🔍 Python will search for modules in: d:\Dev\trading-bot\src


In [ ]:
# Section 6: Complete Model Test Report with Updated Charts
print("📋 Generating Complete Test Report with Updated Interactive Charts...")
print("=" * 70)

# Import our clean testing solution
from src.reporting.model_test_report import quick_test_model

# Run model test - charts will display directly in notebook
print("🚀 Running model test with improved chart design...")

quick_test_model(
    model_dir='models/rl_optimized',
    symbol='BTCUSDT', 
    test_candles=5000  # Use 1500+ for better trade visualizationRecalculating higher timeframe
)

📋 Generating Complete Test Report with Updated Interactive Charts...
🚀 Running model test with improved chart design...
🔍 Testing model: models/rl_optimized
📊 Symbol: BTCUSDT, Candles: 5000
